In [528]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [529]:
import numpy as np
from timer import Timer
import pickle

In [530]:
maxFloat16 = np.finfo(np.float16).max
neuronMax = 1
neuronMin = -1
neuronDtype = np.float32


class NeuralNetwork:
    @staticmethod
    def sigmoid(x, derivative=False):
        if derivative:
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def relu(x, derivative=False):
        if derivative:
            return 1.0 * (x > 0)
        return np.maximum(0, x)

    @staticmethod
    def leakyRelu(x, derivative=False):
        if derivative:
            return 1.0 * (x > 0) + 0.01 * (x <= 0)
        return np.maximum(0.01 * x, x)

    @staticmethod
    def absolute(x, derivative=False):
        if derivative:
            return 1 if x > 0 else -1
        return np.abs(x)


    # @staticmethod
    # def softmax(x):
    #     exps = np.exp(x - np.max(x))
    #     return exps / np.sum(exps, axis=1, keepdims=True)

    # @staticmethod
    # def softmaxDerivative(x):
    #     return x * (1 - x)
    @staticmethod
    def mseLoss(errorVector, derivative=False):
        if derivative:
            return errorVector
        return np.mean(np.square(errorVector))*0.5

    def __init__(self, layerSizes, activationFunctions):
        self.layerSizes = layerSizes
        self.activationFunctions = activationFunctions
        self.layers = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i]).astype(neuronDtype)
            for i in range(len(layerSizes))
        ]
        self.weights = [
            np.random.uniform(
                low=neuronMin, high=neuronMax, size=layerSizes[i] * layerSizes[i + 1]
            ).astype(neuronDtype).reshape(layerSizes[i], layerSizes[i + 1])
            for i in range(len(layerSizes) - 1)
        ]
        self.biases = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i + 1]).astype(
                neuronDtype
            )
            for i in range(len(layerSizes) - 1)
        ]

    def loadInput(self, input):
        self.layers[0] = input.astype(neuronDtype)

    def readOutput(self):
        return self.layers[-1]

    def forward(self):
        for i in range(len(self.weights)):
            self.layers[i + 1] = self.activationFunctions[i](
                np.dot(self.layers[i], self.weights[i]) + self.biases[i]
            )



    def backpropagation(self, target, learning_rate, lossFunction):
        output_error = lossFunction(target - self.layers[-1], derivative=True)

        for i in range(len(self.layers) - 2, -1, -1):
            # Calculate the derivative of the activation function
            derivative = self.activationFunctions[i](self.layers[i + 1], derivative=True)

            # Calculate the error of the current layer
            layer_error = np.dot(output_error * derivative, self.weights[i].T) * learning_rate

            # Calculate the adjustments for the weights and biases
            weight_adjustments = np.outer(self.layers[i], output_error)
            bias_adjustments = output_error

            # Update the weights and biases
            self.weights[i] += weight_adjustments * learning_rate
            self.biases[i] += bias_adjustments * learning_rate

            # Set the output error for the next iteration
            # np.normalize(layer_error)
            output_error = layer_error


In [531]:
x = NeuralNetwork([5_000, 40, 40, 3], [NeuralNetwork.leakyRelu, NeuralNetwork.leakyRelu, NeuralNetwork.leakyRelu])
nnTimer = Timer(precision=3)
target = np.array([2, 3, 4])
input = np.random.uniform(low = 0, high = 1, size = 5_000)

In [532]:
x.loadInput(input)
x.forward()
print(x.readOutput())

[-1.9579115  -0.3506986  -0.81804395]


In [533]:
# learning_rate = 0.00001
# for i in range(200):
#     nnTimer.tic()
#     x.loadInput(input)
#     x.forward()
#     x.backpropagation(target, learning_rate)
#     print(x.readOutput())
#     nnTimer.toc()
# nnTimer.stats()
learning_rate = 0.00001
for i in range(200):
    nnTimer.tic()
    x.loadInput(input)
    x.forward()
    x.backpropagation(target, learning_rate, NeuralNetwork.mseLoss)
    print(x.readOutput(), NeuralNetwork.mseLoss(target - x.readOutput()))
    nnTimer.toc()
nnTimer.stats()

[-1.9579115  -0.3506986  -0.81804395] 8.350965315657668
[-1.9357934  -0.33197355 -0.79111886] 8.257889562484186
[-1.9137983  -0.31335318 -0.76434386] 8.165849846925278
[-1.8919265  -0.29483652 -0.7377187 ] 8.074836381013988
[-1.8701768  -0.27642354 -0.7112425 ] 7.984837584223951
[-1.8485489  -0.25811362 -0.684914  ] 7.895842008525537
[-1.8270413  -0.23990597 -0.65873283] 7.8078378580361365
[-1.805654   -0.22180009 -0.63269776] 7.72081451635016
[-1.7843868  -0.2037951  -0.60680825] 7.634761398037409
[-1.7632381  -0.18589105 -0.5810636 ] 7.549667753205419
[-1.7422076  -0.16808695 -0.5554626 ] 7.46552207585066
[-1.7212945  -0.15038238 -0.5300049 ] 7.382314442245103
[-1.7004985  -0.13277668 -0.50468946] 7.300034277210673
[-1.6798183  -0.11526941 -0.47951534] 7.218670619613818
[-1.6592538  -0.09786015 -0.45448193] 7.138214233954969
[-1.6388044  -0.08054788 -0.42958838] 7.058654363605034
[-1.618469  -0.0633324 -0.4048339] 6.979980850515919
[-1.5982478  -0.04621321 -0.38021794] 6.902185180572